In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import cv2
import sys
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
from PIL import Image
from scipy.spatial import KDTree
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

In [5]:
class LanguageModel():
  def __init__(self):
    self.model = BertModel.from_pretrained('bert-base-uncased',
                                    output_hidden_states = True, # Whether the model returns all hidden-states.
                                    )
    self.model.eval()
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  
  def __tokenize__(self, text):
    start_text = "[CLS] "
    end_text = " [SEP]"

    marked_text = start_text + text + end_text

    tokenized_text = self.tokenizer.tokenize(marked_text)
    # Map the token strings to their vocabulary indeces.
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    

    return tokens_tensor, segments_tensors
  
  def __tokensToEmbeddings__(self, tokens, segments):
    with torch.no_grad():

      outputs = self.model(tokens, segments)
      hidden_states = outputs[2]
  
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs = hidden_states[-2][0]

    sentence_embedding = torch.mean(token_vecs, dim=0)

    return sentence_embedding
  
  def generateEmbeddings(self, text):
    tokens, segments = self.__tokenize__(text)
    embedding = self.__tokensToEmbeddings__(tokens, segments)

    return embedding


class RecommenderSystem():
  def __init__(self, songDF, LanguageModel, EmotionModel):
    self.emotionModel = EmotionModel
    self.LM = LanguageModel
    self.songDF = songDF
    self.songKDTree = None
    self.songLookup = None
    self.__initSongData__()
  
  def __initSongData__(self):
    songsLyrics = self.songDF['caption'].values
    songDF_id = self.songDF['ytid']
    
    songEmbeddings = []

    songLookup = {}
    # i=0
    for (i, song) in enumerate(songsLyrics):
      if i % 100 == 0:
        print("Indexed ", i)
      songEmb = self.LM.generateEmbeddings(song)
      songEmbeddings.append(np.array(songEmb))

      songLookup[tuple(songEmb.tolist())] = songDF_id[i]
    
    songEmbeddings = np.array(songEmbeddings)
    self.songKDTree = KDTree(songEmbeddings)
    self.songLookup = songLookup

  def __imgToEmotion__(self, img):
    return self.emotionModelImg(img)
  
  def __vidToEmotion__(self, vid_path):
    return self.emotionModel.predict_emotions(vid_path)
  
  def __emotionToSongRecs__(self, emotion, n):
    emotionEmb = self.LM.generateEmbeddings(emotion)
    dist, ind = self.songKDTree.query(np.array(emotionEmb), n)

    recs = []

    if n == 1:
      recs.append(self.songLookup[tuple(self.songKDTree.data[ind])])
    else:
      recs = [self.songLookup[tuple(self.songKDTree.data[index])] for index in ind]
    
    return recs
  
  def getRecsFromImage(self, img, n):
    emotion = self.__imgToEmotion__(img)
    songRecs = self.__emotionToSongRecs__(emotion, n)
    
    return songRecs
  
  def getRecsFromVid(self, vid, n):
    emotion = self.__vidToEmotion__(vid)
    songRecs = []

    for emotionPred, _ in emotion:
      songRec = self.__emotionToSongRecs__(emotionPred, n)
      songRecs += songRec  

    return songRecs


class EmotionModel():
  def __init__(self, weights_path, emotion_labels, device):
    self.device = device
    self.emotion_labels = emotion_labels
    self.model = models.vgg19(weights='VGG19_Weights.DEFAULT')
    self.num_features = self.model.classifier[-1].in_features # num of features in last fully connected model layer
    self.model.classifier[-1] = nn.Linear(self.num_features, 7) # 7 unique emotions in both train and test folders resp
    self.model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))

  
  def predict_emotions(self, video_file_path):
    # Load the video
    cap = cv2.VideoCapture(video_file_path)

    # Get the frame rate of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the list of emotions
    emotions = list(self.emotion_labels.keys())
 
    idx_to_class = {v: k for k, v in self.emotion_labels.items()}
    

    # Initialize the emotion count dictionary
    emotion_count = {emotion: 0 for emotion in emotions}

    self.model.to(self.device)

    # Loop over the frames of the video
    while cap.isOpened():
        # Read the frame
        ret, frame = cap.read()

        # Check if the frame was successfully read
        if not ret:
            break

        # Preprocess the frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        # Preprocess the frame
        frame = transforms.Grayscale(num_output_channels=3)(frame) # Pre-trained model expects RGB
        frame = transforms.Resize((224, 224))(frame) # Want a small standard image size to reduce load on GPU
        frame = transforms.ToTensor()(frame)
        frame = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])(frame) # Normalize(mean_pixel_val, std_pixel_val)
        frame = frame.unsqueeze(0)

        frame = frame.to(self.device)

        # Pass the frame through the model to obtain the predicted emotion
        with torch.no_grad():
            outputs = self.model(frame)
            _, predicted = torch.max(outputs.data, 1)
            predicted_emotion = idx_to_class[predicted.item()]

        # Increment the count for the predicted emotion
        emotion_count[predicted_emotion] += 1

        # Wait for the specified amount of time before processing the next frame
        cv2.waitKey(1000 // fps)

    # Release the video capture
    cap.release()

    # Get the top 3 predicted emotions
    top_3_emotions = sorted(emotion_count.items(), key=lambda x: x[1], reverse=True)[:3]

    return top_3_emotions




def main(video_path, n):
  device = 'cpu'
  emotion_dict = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
  songDF = pd.read_csv("/content/drive/Shareddrives/AML Project/musiccaps-public.csv")
  LM = LanguageModel()
  EM = EmotionModel('/content/drive/Shareddrives/AML Project/CV_PART/vgg_modified.pth', emotion_dict, device)
  RS = RecommenderSystem(songDF, LanguageModel= LM, EmotionModel= EM)

  print(RS.getRecsFromVid(video_path, n=n))

In [ ]:
if __name__ == '__main__':
  vid_path = input("video path: ")
  n = input("number of recommendations: ")
  main(vid_path, n)